<a href="https://colab.research.google.com/github/perlatomdpi/End-Point/blob/main/3_Create_Flask_App_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Create Flask Application - Part 1**



In [ ]:
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonifly   # we use json to return data in the flask app
import os
import numpy as np
import uuid  # identifier

app = Flask(__name__)  # create flask application

In [ ]:
# create checks in our application 
# to be sure that those  checks are met in order to make a prediction right with no errors
# these checks ensure us that the end-user will not get any errors
# to esure this, we create the "Expected" variable with the boundary values of each predictors

Expected = {
"cylinders":{"min":3,"max":8},
"displacement":{"min":68.0,"max":455.0},
"horsepower":{"min":46.0,"max":230.0},
"weight":{"min":1613,"max":5140},
"acceleration":{"min":8.0,"max":24.8},
"year":{"min":70,"max":82},
"origiin":{"min":1,"max":3},
}

In [ ]:
# load the model
os.chdir(r'C:\User...\Deploy')  
model = load_model(os.path.join(os.getcwd(), "mpg_model.h5"))

model.summary()   # check if the momdel is available

# **Create Flask Application - Part 2**
Create Flask end-point and ensure data quality checks.

In [ ]:
# define the end point
@app.route('/api', methods=['POST'])   # send the data to the end point, and define the method --> post (send) data to the end point

def mpg_prediction():
    content = request.json    # get json data --> similar to a python dictionary
    errors = []               # list of errors
    for name in content:
        if name in Expected:  
            expected_min = Expected[name]['min']
            expected_max = Expected[name]['max']
            value = content[name]
            if value < expected_min or value > expected_max:   # ensure to get valid data
                errors.append(f"Out of bounds:{name}, has a value of:{value}, but it should be between {expected_min} and {expected_max}")
        else:
            errors.append(f"Unexpected field: {name}")
    for name in Expected:
        if name not in content:
            errors.append(f"Missing value:{name}")
    if len(errors)<1:                                          # if there are not errors
        x = np.zeros((1,7))                                    # create a zeros array with 1 rowa dn 7 columns --> 7 are our predictors of mpg
        x[0,0] = content['cylinders']
        x[0,1] = content['displacement']
        x[0,2] = content['horsepower']
        x[0,3] = content['weight']
        x[0,4] = content['acceleration']
        x[0,5] = content['year']
        x[0,6] = content['origin']

        predict = model.predict(x)
        mpg = float(prediction[0])
        response ={"id": str(uuid4.uuid4()), "mpg": mpg, "errors":errors}      # "id": str(uuid4.uuid4()) --> refers to unique id
    else:                                                       # if there are errors -- len(errors)>1
        response = {"id": str(uuid4.uuid4()), "errors":errors}

    return jsonify(response)


# run application
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)    # debag is true because we are testing the app

Service Flask app "__main__" (lazy loading) <br>
Environment: priduction <br>
Warining: Do not use the development server in a production environment. <br>
Use a production WSGI server instead. <br>
Debug mode: on 

# **Create Flask Application - Part 3**
Testing our API endpoint. <br>
Making prediction using Python and Requests library.

In [ ]:
# Making prediction using Python and Requests library
import requests

json = {"cylinders":8,               # dictionary used as input prediction values
       "displacement":{200,
       "horsepower":80,
       "weight":{2500,
       "acceleration":30,
       "year":{"min":80,
       "origiin":3}

r = request.post("http://localhost:5000/api", json=json)  # the defalut flask port is 5000 and the end point we preaviously defined is "api"

if r.status_code == 200:  # 200 means --> that the request if fine
    print("Works: {}".format(r.text))
else: print("Fail: {}".format(r.text))

Making prediction using Postman. <br>

We can also use tools sich as Postman that allows to make requests: <br> 
1 - POST: http://127.0.0.1:500/api <br>
2 - Define inside the headers a content type --> applicaition/json --> set postman to send json data <br>
3 - Defne the body: row data in json

